In [19]:
import pandas as pd
import crew_optimizer
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

In [20]:
assignment = pd.read_json("assignments/assignment.json")
assignment = assignment['assignment'][4:254]
assignment

FR889    {'captain': 'C42', 'first_officer': 'FO21', 'd...
FR408    {'captain': 'C21', 'first_officer': 'FO7', 'de...
FR120    {'captain': None, 'first_officer': 'FO42', 'de...
FR306    {'captain': 'C45', 'first_officer': 'FO0', 'de...
FR141    {'captain': 'C4', 'first_officer': 'FO12', 'de...
                               ...                        
C28                                                    NaN
C29                                                    NaN
C30                                                    NaN
C31                                                    NaN
C32                                                    NaN
Name: assignment, Length: 250, dtype: object

In [21]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
print(f"Using OpenAI API Key: {openai_api_key[:4]}...{openai_api_key[-4:]}")

Using OpenAI API Key: sk-A...dosP


In [22]:
instruction = f"""You are a large neighbourhood search (LNS) optimisation algorithm designer working
with an expert scheduler for an airline operation. Your task is to design a set of LNS destroy
and repair operators that will assign captains and first-officers to flights based on their
qualifications and availability. Each crew is certifications that determine which aircraft they can
fly. You must ensure that all shifts are adequately staffed while adhering to the following
constraints:
1. Each flight must have one captain and one first-officer.
2. Captains can only be assigned to flights that match their certifications.
3. First-officers can only be assigned to flights that match their certifications.
4. Crew members cannot be assigned to overlapping flights.
5. Each crew member has a maximum number of hours they can work per week.
6. Try to balance the workload among crew members as evenly as possible.
Analyse the assignment data and propose a single destroy or repair operator that could be used
in the LNS algorithm to tackle the worst issue in the current assignment. Provide a detailed
description of how the operator works, including its logic and steps. Your response should be in
the following JSON format:
""
{{
  "operator_type": "destroy" or "repair",
  "description": "Detailed description of the operator logic and steps"
}}
"""

In [24]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": instruction},
        {
            "role": "user",
            "content": f"Here is the current assignment data: {assignment.to_json(orient='records')}",
        },
    ],
    temperature=0.7,
    max_tokens=1000,
    n=1,
    stop=None,
 )

response_content = response.choices[0].message.content
print("Response from OpenAI:")
print(response_content)

#operator_suggestion = json.loads(response_content)
#print(operator_suggestion)

Response from OpenAI:
{
  "operator_type": "repair",
  "description": "From the given assignment data, the worst issue seems to be the presence of several flights without assigned captains and first officers. This repair operator will aim to address this issue. The logic of the operator works as follows: 1. Identify all flights where either the captain or the first officer (or both) are not assigned. 2. For each of these flights, check the certifications and availability of the unassigned crew members. 3. Try to assign a captain and first officer to the flight from the available crew members based on their certifications. If multiple crew members fit the requirements, preference will be given to those who have the least number of shifts assigned to them. 4. If a suitable captain or first officer cannot be found, the algorithm will attempt to reassign crew members from other flights, provided this does not lead to any violations of the constraints. The aim is to ensure that all flights 